In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [5]:
A = pd.read_csv("../assets/data/cleaned/A.csv", index_col=0)
B = pd.read_csv("../assets/data/cleaned/B.csv", index_col=0)
C = pd.read_csv("../assets/data/cleaned/C.csv", index_col=0)
D = pd.read_csv("../assets/data/cleaned/D.csv", index_col=0)
E = pd.read_csv("../assets/data/cleaned/E.csv", index_col=0)

In [62]:
def train_test_val_split(X, y, test_size, val_size=0.4):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=val_size)
    return X_train, X_test, X_val, y_train, y_test, y_val

def reshapeInput(X):
    X_ = []
    for x in X:
        X_.append(x.reshape(-1,1))
    return np.array(X_)

def trend(X, span = None):
    '''
    Return EWMA trend for signal samples
    '''
    if not span:
        span = X.shape[1]
    return pd.DataFrame(X).apply(lambda x: x.ewm(span=span).mean(), axis=1)

In [129]:
X = pd.concat([A,B,C,D,E], axis=0)
Y = [0]*A.__len__() + [1]*B.__len__() + [2]*C.__len__() + [3]*D.__len__() + [4]*E.__len__()

In [130]:
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, tf.keras.utils.to_categorical(Y), test_size=0.2)

In [131]:
np.sum(y_train, axis=0)

array([ 970.,  901., 1054.,  808., 1123.], dtype=float32)

In [133]:
X_train, y_train = RandomUnderSampler(random_state=42).fit_resample(X_train, y_train)

C:\Users\sabri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [134]:
np.sum(y_train, axis=0)

array([808, 808, 808, 808, 808])

In [135]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20), input_shape=(24, 1)))
model.add(tf.keras.layers.Dense(12, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [153]:
model.fit(reshapeInput(X_train.values), y_train, 
          epochs=120, batch_size=20,
          validation_data=(reshapeInput(X_val.values), y_val),
         callbacks=tf.keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True))

Epoch 1/120
202/202 [==============================] - 2s 8ms/step - loss: 0.6689 - accuracy: 0.6876 - val_loss: 0.6947 - val_accuracy: 0.6770
Epoch 2/120
202/202 [==============================] - 2s 8ms/step - loss: 0.6727 - accuracy: 0.6933 - val_loss: 0.7252 - val_accuracy: 0.6584
Epoch 3/120
202/202 [==============================] - 2s 8ms/step - loss: 0.6663 - accuracy: 0.7012 - val_loss: 0.6776 - val_accuracy: 0.6831
Epoch 4/120
202/202 [==============================] - 2s 8ms/step - loss: 0.6857 - accuracy: 0.7012 - val_loss: 0.6932 - val_accuracy: 0.6584
Epoch 5/120
202/202 [==============================] - 2s 8ms/step - loss: 0.6739 - accuracy: 0.6928 - val_loss: 0.7731 - val_accuracy: 0.6605
Epoch 6/120
202/202 [==============================] - 2s 8ms/step - loss: 0.6653 - accuracy: 0.7074 - val_loss: 0.7238 - val_accuracy: 0.6543
Epoch 7/120
202/202 [==============================] - 2s 8ms/step - loss: 0.6744 - accuracy: 0.6978 - val_loss: 0.6827 - val_accuracy: 0.6646

In [154]:
print(confusion_matrix(np.argmax(y_test, axis=1),model.predict_classes(reshapeInput(X_test.values))))

[[125  26   3   5   0]
 [ 33  58   9  15   4]
 [  4   9 140  21   0]
 [  1  23  10  77  19]
 [  1  10   1  22 112]]


C:\Users\sabri\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [155]:
print(accuracy_score(np.argmax(y_test, axis=1),model.predict_classes(reshapeInput(X_test.values))))

0.7032967032967034


C:\Users\sabri\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


model.save("../assets/models/finalABCDE")

<b>
1. So the final model has an accuracy of 70 %.<br/><br/>
2. This was achieved primarily using data cleaning; Given more time, I would have liked to research dsp technique effectiveness instead of using a temporal modeling approach.<br/><br/>
3. We just need to load the model, and since the model learnt on cleaner data, we dont need to perform any feature extraction pipelines and can process on the input signal.<br/><br/>
4. We can use the autoencoders to first understand the signal pattern and bucket the signal; this can be used to re-inforce the model prediction.<br/><br/>
5. Lastly this was a lot of fun😁. Thanks.<br/>
</b>